In [15]:
import cv2
import os
import torch
import keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output
import glob

In [8]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
model.classes = 16

clear_output()

In [4]:
PATH_INIT = "./data/DogCategories/"
PATH_SAVE = "./data/DogCategories_cropped/"
dir_list = os.listdir(PATH_INIT)
dir_list

['.ipynb_checkpoints',
 'DOG ONLY - multicolored - long tail',
 'DOG ONLY - multicolored - short tail',
 'DOG ONLY DARK LONG',
 'DOG ONLY DARK SHORT',
 'DOG ONLY LIGHT LONG',
 'DOG ONLY LIGHT SHORT']

In [22]:
PATH_INIT = "./data/DogCategories/"
PATH_SAVE = "./data/DogCategories_cropped/"
dir_list = os.listdir(PATH_INIT)
shapes = []
for category in dir_list:
    if category == '.ipynb_checkpoints':
        continue
    
    category_dir = os.path.join(PATH_INIT, category)
    category_save_dir = os.path.join(PATH_SAVE, category)
    
    images_list = os.listdir(category_dir)
    images_list = list(filter(lambda x: x.endswith('.jpg') or x.endswith('.png'), images_list))
    
    for image_name in images_list:
        try:
            full_image_name = os.path.join(category_dir, image_name)
            image = cv2.imread(full_image_name)
            predict_result = model(full_image_name)
        except Exception as e:
            continue
        
        bb_doglist = eval(predict_result.pandas().xyxy[0].to_json(orient="records"))
        
        if len(bb_doglist) == 0:
            continue
        elif len(bb_doglist) == 1:
            image_cropped = image[int(bb_doglist[0]['ymin']):int(bb_doglist[0]['ymax']),int(bb_doglist[0]['xmin']):int(bb_doglist[0]['xmax'])]
            plt.imsave(f"{category_save_dir}/{image_name}", image_cropped)
            
            flip_image_cropped = cv2.flip(image_cropped, 1)
            plt.imsave(f"{category_save_dir}/{image_name[:-4]}_fliped{image_name[-4:]}", flip_image_cropped)
        else:
            counter = 1
            for bb_dog in bb_doglist:
                image_cropped = image[int(bb_dog['ymin']):int(bb_dog['ymax']),int(bb_dog['xmin']):int(bb_dog['xmax'])]
                plt.imsave(f"{category_save_dir}/{image_name[:-4]}_{counter}{image_name[-4:]}", image_cropped)
                
                flip_image_cropped = cv2.flip(image_cropped, 1)
                plt.imsave(f"{category_save_dir}/{image_name[:-4]}_fliped_{counter}{image_name[-4:]}", flip_image_cropped)
                
                counter += 1
        shapes.append(image_cropped.shape)

In [18]:
np.mean(shapes, axis=0)

array([     59.975,      70.172,           3])

In [27]:
img = "./data/Experiment/JustSomeDog/138.jpg"
results = model(img)

In [28]:
eval(results.pandas().xyxy[0].to_json(orient="records"))

[{'xmin': 428.4515686035,
  'ymin': 581.0015258789,
  'xmax': 457.1656188965,
  'ymax': 615.214050293,
  'confidence': 0.2609863281,
  'class': 16,
  'name': 'dog'}]